In [1]:
from datasets import load_dataset
import os
import json 

livecodebench = load_dataset("livecodebench/code_generation_lite", version_tag="release_v5", trust_remote_code=True, split="test")
print(livecodebench)

/opt/conda/envs/rllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 880 examples [00:26, 33.39 examples/s] 

Dataset({
    features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
    num_rows: 880
})


In [2]:
from datetime import datetime

# deepseek paper uses lcb from 2024-08 to 2025-01
def is_date_in_range(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S")
    # Define the start and end range
    start_date = datetime(2024, 8, 1)  # August 1, 2024
    end_date = datetime(2025, 2, 1)    # February 1, 2025

    # Check if the date is within the range
    return start_date <= date_obj < end_date

In [5]:
dataset = []
for entry in livecodebench:
    tests = entry["public_test_cases"]
    if isinstance(tests, str):
        try:
            tests = json.loads(tests)
        except json.JSONDecodeError as e:
            print(f"code reward Json Error parsing livecodebench: {e}")
            continue 

    if len(tests) == 0:
        continue 
    new_entry = {
        "problem": entry["question_content"],
        "starter_code": entry["starter_code"],
        "tests": tests,
    }
    dataset.append(new_entry)

print(f'len(dataset): {len(dataset)}')

output_dir = os.path.abspath("../../test/code")
output_file = os.path.join(output_dir, "livecodebench.json")

with open(output_file, "w") as f:
    json.dump(dataset, f, indent=4)


{'contest_date': '2023-09-03T00:00:00',
 'contest_id': 'weekly-contest-361',
 'difficulty': 'medium',
 'metadata': '{"func_name": "minimumOperations"}',
 'platform': 'leetcode',
 'private_test_cases': 'eJy1kjtKBEEQQA02M/MEQ8eL1P/jSQTXQGSFBZldcCYQETyE3tcaMdEBYRisqLuqX/crut42Hxebs6+4Pq/FzUs79KdxaFdd2zXftbbt2nEcvlM4bYf90zA8n/ZT4mHs74fDsb97bK/b7gfMa2BaA+MaONbAuQb2GU1LvGfiS2ifffUiWlXEQRNULYFBkdKRSdGBlasOxOZmbGqelqKEVAWpc6IsDCYckMKqHB5kVoyEBBMKYc6/ZZFguIZzWVndplErctZQd/CpZCAkARFGxJlRb2sqAwGZA2q6G2PdINUaVraa8HITy+q9pHE+7rBkZOHfg/S34Z+Ct++Xn7G4HPw=',
 'public_test_cases': '[{"input": "\\"2245047\\"", "output": "2", "testtype": '
                      '"functional"}, {"input": "\\"2908305\\"", "output": '
                      '"3", "testtype": "functional"}, {"input": "\\"10\\"", '
                      '"output": "1", "testtype": "functional"}]',
 'question_content': 'You are given a 0-indexed string num representing a '
                     'non-negative integer.\n'
                     'In 